<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/train_hyperdim_ai_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch numpy matplotlib

In [ ]:
#!/usr/bin/env python3
"""
train_hyperdim_ai.py

Physics‐informed AI pipeline for mapping 5D coordinates to 3D navigation metrics:

 1. Synthetic dataset of 5D points → 3D analytic navigation factors
 2. PINN loss: MSE + hyperdistance‐preserving residual
 3. MLP with LayerNorm & Dropout for uncertainty estimation
 4. MC‐Dropout inference to quantify predictive variance
 5. Training loop with AdamW, ReduceLROnPlateau, early stopping
 6. Visualizations: loss curves, true vs predicted, uncertainty heatmap
"""

import math
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt

# ------------------------------------------------------------------------------
# 1. Synthetic Dataset
# ------------------------------------------------------------------------------
class HyperDimDataset(Dataset):
    def __init__(self, n_samples=6000, seed=0):
        np.random.seed(seed)
        # Sample 5D coordinates in [-π, π]
        X_raw = np.random.uniform(-math.pi, math.pi, size=(n_samples, 5)).astype(np.float32)
        # Analytic 3D navigation factors
        #   y0 = sin(x0) + cos(x1)
        #   y1 = x2 * x3
        #   y2 = exp(−x4²)
        y0 = np.sin(X_raw[:, 0]) + np.cos(X_raw[:, 1])
        y1 = X_raw[:, 2] * X_raw[:, 3]
        y2 = np.exp(- X_raw[:, 4]**2)
        Y_raw = np.stack([y0, y1, y2], axis=1).astype(np.float32)
        # add noise
        Y_raw += 0.02 * np.random.randn(*Y_raw.shape).astype(np.float32)

        # compute normalization stats
        self.X_mean, self.X_std = X_raw.mean(0), X_raw.std(0)
        self.Y_mean, self.Y_std = Y_raw.mean(0), Y_raw.std(0)

        # normalize
        self.X = (X_raw - self.X_mean) / self.X_std
        self.Y = (Y_raw - self.Y_mean) / self.Y_std

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = torch.from_numpy(self.X[idx])
        y = torch.from_numpy(self.Y[idx])
        return x, y

# ------------------------------------------------------------------------------
# 2. Model Architecture with Dropout
# ------------------------------------------------------------------------------
class HyperDimAI(nn.Module):
    def __init__(self, input_dim=5, hidden_dims=(64,64), output_dim=3, p_drop=0.1):
        super().__init__()
        layers, dim = [], input_dim
        for h in hidden_dims:
            layers += [
                nn.Linear(dim, h),
                nn.LayerNorm(h),
                nn.ReLU(),
                nn.Dropout(p_drop)
            ]
            dim = h
        layers.append(nn.Linear(dim, output_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

# ------------------------------------------------------------------------------
# 3. Physics-Informed Loss: Hyperdistance Preservation
# ------------------------------------------------------------------------------
def physics_residual(pred, inp, stats_torch):
    # denormalize
    X_den = inp * stats_torch['X_std'] + stats_torch['X_mean']
    Y_den = pred * stats_torch['Y_std'] + stats_torch['Y_mean']
    # compute squared norms
    norm2_X = torch.sum(X_den**2, dim=1)
    norm2_Y = torch.sum(Y_den**2, dim=1)
    # residual: ||X||² ≈ ||Y||²
    return torch.mean((norm2_X - norm2_Y)**2)

def total_loss(pred, true, inp, stats_torch, lambda_phys=0.5):
    mse  = nn.MSELoss()(pred, true)
    phys = physics_residual(pred, inp, stats_torch)
    return mse + lambda_phys * phys, mse, phys

# ------------------------------------------------------------------------------
# 4. MC-Dropout Inference for Uncertainty
# ------------------------------------------------------------------------------
def mc_dropout_predict(model, x, n_samples=100):
    model.train()
    preds = []
    with torch.no_grad():
        for _ in range(n_samples):
            preds.append(model(x).cpu().numpy())
    arr = np.stack(preds, axis=0)
    return arr.mean(axis=0), arr.std(axis=0)

# ------------------------------------------------------------------------------
# 5. Training Loop
# ------------------------------------------------------------------------------
def train(model, train_loader, val_loader, stats_torch, device,
          lr=1e-3, wd=1e-5, lambda_phys=0.5,
          max_epochs=100, patience=10):
    model.to(device)
    opt = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    sched = optim.lr_scheduler.ReduceLROnPlateau(opt, mode='min',
                                                factor=0.5, patience=5)

    best_val, wait = float('inf'), 0
    history = {'train_loss': [], 'val_loss': []}

    for epoch in range(1, max_epochs+1):
        # training
        model.train()
        tr_loss = 0.0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            loss, _, _ = total_loss(pred, yb, xb, stats_torch, lambda_phys)
            opt.zero_grad()
            loss.backward()
            opt.step()
            tr_loss += loss.item() * xb.size(0)
        tr_loss /= len(train_loader.dataset)

        # validation
        model.eval()
        vl_loss = 0.0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                pred = model(xb)
                loss, _, _ = total_loss(pred, yb, xb, stats_torch, lambda_phys)
                vl_loss += loss.item() * xb.size(0)
        vl_loss /= len(val_loader.dataset)

        sched.step(vl_loss)
        history['train_loss'].append(tr_loss)
        history['val_loss'].append(vl_loss)
        print(f"Epoch {epoch:03d} | Train {tr_loss:.4e} | Val {vl_loss:.4e}")

        if vl_loss < best_val - 1e-6:
            best_val, wait = vl_loss, 0
            torch.save(model.state_dict(), "best_hyperdim_ai.pth")
        else:
            wait += 1
            if wait >= patience:
                print(f"Early stopping at epoch {epoch}")
                break

    model.load_state_dict(torch.load("best_hyperdim_ai.pth"))
    return history

# ------------------------------------------------------------------------------
# 6. Visualization Helpers
# ------------------------------------------------------------------------------
def plot_losses(history):
    plt.figure()
    plt.plot(history['train_loss'], label='Train')
    plt.plot(history['val_loss'],   label='Val')
    plt.xlabel("Epoch"); plt.ylabel("Loss")
    plt.legend(); plt.title("Training Curve")
    plt.show()

def plot_scatter(true_vals, pred_vals, name):
    plt.figure()
    plt.scatter(true_vals, pred_vals, s=5, alpha=0.5)
    m, M = true_vals.min(), true_vals.max()
    plt.plot([m, M], [m, M], 'r--')
    plt.xlabel(f"True {name}"); plt.ylabel(f"Pred {name}")
    plt.title(f"{name}: True vs Pred")
    plt.show()

def plot_uncertainty_heatmap(model, stats_np, device):
    # vary x0, x1 in [-π,π], fix others at 0
    vals = np.linspace(-math.pi, math.pi, 100)
    G0, G1 = np.meshgrid(vals, vals)
    grid = np.zeros((G0.size, 5), dtype=np.float32)
    grid[:,0] = G0.ravel()
    grid[:,1] = G1.ravel()

    # normalize
    Xn = (grid - stats_np['X_mean']) / stats_np['X_std']
    Xt = torch.from_numpy(Xn).float().to(device)
    _, std = mc_dropout_predict(model, Xt, n_samples=100)
    # take std of first output
    std_map = std[:,0].reshape(G0.shape)

    plt.figure(figsize=(6,5))
    plt.pcolormesh(G0, G1, std_map, shading='auto', cmap='viridis')
    plt.colorbar(label="Std of y0")
    plt.xlabel("x0"); plt.ylabel("x1")
    plt.title("Uncertainty Heatmap for y0")
    plt.show()

# ------------------------------------------------------------------------------
# 7. Main Execution
# ------------------------------------------------------------------------------
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # prepare data
    ds = HyperDimDataset(n_samples=8000)
    stats_torch = {
        'X_mean': torch.tensor(ds.X_mean, device=device),
        'X_std':  torch.tensor(ds.X_std,  device=device),
        'Y_mean': torch.tensor(ds.Y_mean, device=device),
        'Y_std':  torch.tensor(ds.Y_std,  device=device),
    }
    stats_np = {
        'X_mean': ds.X_mean,
        'X_std':  ds.X_std,
        'Y_mean': ds.Y_mean,
        'Y_std':  ds.Y_std,
    }

    # split
    n_val = int(0.2 * len(ds))
    trn_ds, val_ds = random_split(ds, [len(ds)-n_val, n_val])
    trn_ld = DataLoader(trn_ds, batch_size=64, shuffle=True)
    val_ld = DataLoader(val_ds,   batch_size=128)

    # build & train
    model   = HyperDimAI().to(device)
    history = train(model, trn_ld, val_ld, stats_torch, device)

    # plots
    plot_losses(history)

    # scatter for each output
    X_all = ds.X
    with torch.no_grad():
        Y_pred_norm = model(torch.from_numpy(X_all).float().to(device)).cpu().numpy()
    Y_true_norm = ds.Y
    Y_pred = Y_pred_norm * ds.Y_std + ds.Y_mean
    Y_true = Y_true_norm * ds.Y_std + ds.Y_mean

    for i, name in enumerate(["y0","y1","y2"]):
        plot_scatter(Y_true[:,i], Y_pred[:,i], name)

    # uncertainty heatmap
    plot_uncertainty_heatmap(model, stats_np, device)